In [20]:
%%writefile train_partB.py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import wandb
import argparse
import os

os.environ['WANDB_API_KEY'] = '1ffc33d77af0fd022201ec32b81cd0e92cd75821'
wandb.login()

class ResNetFineTuner(nn.Module):
    def __init__(self, config, num_classes=10):
        super(ResNetFineTuner, self).__init__()
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        for param in self.model.parameters():
            param.requires_grad = False

        modules = []
        if config.dropout > 0:
            modules.append(nn.Dropout(config.dropout))
        modules.append(nn.Linear(self.model.fc.in_features, config.num_dense))

        if config.batch_norm == "true":
            modules.append(nn.BatchNorm1d(config.num_dense))

        modules.append(self.get_activation(config.activation))
        modules.append(nn.Linear(config.num_dense, num_classes))
        self.model.fc = nn.Sequential(*modules)

        for param in self.model.fc.parameters():
            param.requires_grad = True

        self.model = self.model.to(self.device)
        self.criterion = nn.CrossEntropyLoss()
        if config.optimizer == "adam":
            self.optimizer = optim.Adam(self.model.parameters(), lr=config.lr)
        elif config.optimizer == "nadam":
            self.optimizer = optim.NAdam(self.model.parameters(), lr=config.lr)

        self.train_loader, self.val_loader = self.get_data_loaders()

    def get_activation(self, name):
        return {
            "ReLU": nn.ReLU(),
            "GELU": nn.GELU(),
            "SiLU": nn.SiLU(),
            "Mish": nn.Mish()
        }.get(name, nn.ReLU())

    def get_data_loaders(self):
        mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
        train_transforms = [transforms.Resize((224, 224))]
        if self.config.data_aug == "true":
            train_transforms += [transforms.RandomHorizontalFlip(), transforms.RandomRotation(15)]
        train_transforms += [transforms.ToTensor(), transforms.Normalize(mean, std)]

        val_transforms = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean, std),
        ])

        train_dataset = datasets.ImageFolder("/kaggle/input/dl-assignment-2/inaturalist_12K/train",
                                             transform=transforms.Compose(train_transforms))
        val_dataset = datasets.ImageFolder("/kaggle/input/dl-assignment-2/inaturalist_12K/val",
                                           transform=val_transforms)

        train_loader = DataLoader(train_dataset, batch_size=self.config.batch_size, shuffle=True, num_workers=2)
        val_loader = DataLoader(val_dataset, batch_size=self.config.batch_size, shuffle=False, num_workers=2)

        return train_loader, val_loader

    def train_model(self, num_epochs=10):
        for epoch in range(num_epochs):
            self.model.train()
            running_loss, correct, total = 0, 0, 0
            for images, labels in self.train_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

            train_loss = running_loss / total
            train_acc = correct / total

            val_loss, val_acc = self.validate()
            wandb.log({"train_loss": train_loss, "train_acc": train_acc * 100, "val_loss": val_loss, "val_acc": val_acc * 100})

            print(f"Epoch {epoch + 1}: Train Acc: {train_acc * 100:.2f}%, Val Acc: {val_acc * 100:.2f}%, Val Loss: {val_loss:.4f}")

        torch.save(self.model.state_dict(), "finetuned_resnet50.pth")

    def validate(self):
        self.model.eval()
        val_loss = 0
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for images, labels in self.val_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        return val_loss / val_total, val_correct / val_total


def parse_args():
    parser = argparse.ArgumentParser(description="Fine-tune ResNet50 with configurable options and W&B logging")
    parser.add_argument("-wp", "--wandb_project", type=str, required=True)
    parser.add_argument("-we", "--wandb_entity", type=str, required=True)
    parser.add_argument("--dropout", type=float, default=0.3)
    parser.add_argument("--lr", type=float, default=0.001)
    parser.add_argument("--activation", type=str, default="Mish", choices=["ReLU", "GELU", "SiLU", "Mish"])
    parser.add_argument("--optimizer", type=str, default="adam", choices=["adam", "nadam"])
    parser.add_argument("--batch_norm", type=str, default="true", choices=["true", "false"])
    parser.add_argument("--data_aug", type=str, default="true", choices=["true", "false"])
    parser.add_argument("--batch_size", type=int, default=128)
    parser.add_argument("--num_dense", type=int, default=128)
    parser.add_argument("--epochs", type=int, default=10)
    # return vars(parser.parse_args())
    return parser.parse_args()


def main():
    # args = parse_args()

    # wandb.init(project=args["wandb_project"], entity=args["wandb_entity"], config=args)
    args = parse_args()

    wandb.init(project=args.wandb_project, entity=args.wandb_entity, config=vars(args))
    wandb.run.name = (
        f"{args.optimizer}-{args.activation}-bn_{args.batch_norm}-da_{args.data_aug}-do_{args.dropout}"
        f"-bs_{args.batch_size}-lr_{args.lr}-fc_{args.num_dense}"
    )

    model = ResNetFineTuner(args)
    model.train_model(num_epochs=args.epochs)
    wandb.finish()


if __name__ == "__main__":
    main()


Overwriting train_partB.py


In [21]:
!python train_partB.py --wandb_entity alandandoor-iit-madras --wandb_project DL_A2

wandb: Currently logged in as: alandandoor (alandandoor-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Tracking run with wandb version 0.19.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250419_153817-nalnlwew
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run rich-pond-585
wandb: ⭐️ View project at https://wandb.ai/alandandoor-iit-madras/DL_A2
wandb: 🚀 View run at https://wandb.ai/alandandoor-iit-madras/DL_A2/runs/nalnlwew
Epoch 1: Train Acc: 65.27%, Val Acc: 72.75%, Val Loss: 0.7989
Epoch 2: Train Acc: 71.38%, Val Acc: 72.15%, Val Loss: 0.7944
Epoch 3: Train Acc: 72.32%, Val Acc: 74.40%, Val Loss: 0.7519
Epoch 4: Train Acc: 72.93%, Val Acc: 74.45%, Val Loss: 0.7266
Epoch 5: Train Acc: 74.60%, Val Acc: 75.60%, Val Loss: 0.7312
Epoch 6: Train Acc: 74.69%, Val Acc: 75.20%, Val Loss: 0.7386
Epoch 7: Tra

In [22]:
!python train_partB.py --wandb_entity alandandoor-iit-madras --wandb_project DL_A2 --dropout 0.2 --lr 0.0001 --activation Mish --optimizer nadam --batch_norm true --data_aug true --batch_size 64 --num_dense 128


wandb: Currently logged in as: alandandoor (alandandoor-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Tracking run with wandb version 0.19.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250419_155113-bklaspes
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run crimson-mountain-586
wandb: ⭐️ View project at https://wandb.ai/alandandoor-iit-madras/DL_A2
wandb: 🚀 View run at https://wandb.ai/alandandoor-iit-madras/DL_A2/runs/bklaspes
Epoch 1: Train Acc: 56.96%, Val Acc: 70.65%, Val Loss: 1.0309
Epoch 2: Train Acc: 68.08%, Val Acc: 72.90%, Val Loss: 0.8839
Epoch 3: Train Acc: 70.94%, Val Acc: 73.40%, Val Loss: 0.8306
Epoch 4: Train Acc: 71.79%, Val Acc: 73.85%, Val Loss: 0.7973
Epoch 5: Train Acc: 72.75%, Val Acc: 73.70%, Val Loss: 0.7865
Epoch 6: Train Acc: 73.05%, Val Acc: 75.20%, Val Loss: 0.7611
Epoch